In [2]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')

In [4]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X_train = train.drop(['label'], axis=1)
y_train = train['label']

In [6]:
X_train.shape

(42000, 784)

In [7]:
y_train.shape

(42000,)

In [8]:
# 欠損値確認
X_train.isna().any().describe()

count       784
unique        1
top       False
freq        784
dtype: object

In [9]:
test.isna().any().describe()

count       784
unique        1
top       False
freq        784
dtype: object

In [10]:
X_train /= 255.0
test /= 255.0

In [11]:
X_train = X_train.values.reshape(-1, 28, 28, 1)
test = test.values.reshape(-1, 28, 28, 1)

In [12]:
X_train.shape

(42000, 28, 28, 1)

In [13]:
from keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes=10)

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [15]:
X_train.shape

(33600, 28, 28, 1)

In [16]:
y_train.shape

(33600, 10)

In [17]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

Metal device set to: Apple M2

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [18]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 128)         73856 

In [19]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [20]:
epochs = 3
batch_size = 64

In [21]:
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_validate, y_validate))

Epoch 1/3


2023-04-05 19:05:09.620649: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


525/525 [==============================] - 7s 13ms/step - loss: 0.2441 - accuracy: 0.9253 - val_loss: 0.1009 - val_accuracy: 0.9692
Epoch 2/3
525/525 [==============================] - 6s 12ms/step - loss: 0.0588 - accuracy: 0.9810 - val_loss: 0.0559 - val_accuracy: 0.9818
Epoch 3/3
525/525 [==============================] - 6s 12ms/step - loss: 0.0419 - accuracy: 0.9870 - val_loss: 0.0517 - val_accuracy: 0.9839


In [22]:
results = model.predict(test)
results

875/875 [==============================] - 2s 2ms/step


array([[3.23959100e-08, 2.04467767e-08, 9.99982834e-01, ...,
        1.45312524e-05, 1.61152755e-07, 3.83690724e-09],
       [9.99111235e-01, 2.13342211e-09, 1.61627995e-05, ...,
        1.24111475e-05, 2.43904765e-06, 1.46191451e-04],
       [1.99100464e-06, 1.63373090e-06, 4.75807101e-05, ...,
        3.28638300e-04, 4.38523432e-03, 9.94254410e-01],
       ...,
       [4.86781448e-12, 9.25425125e-10, 1.82891835e-09, ...,
        1.19477654e-08, 1.98092039e-07, 3.68499116e-07],
       [2.73379754e-07, 5.71390729e-07, 5.15333909e-07, ...,
        3.21865431e-04, 1.64437060e-05, 9.92893875e-01],
       [9.14205671e-08, 1.01931148e-08, 9.99993205e-01, ...,
        1.69661269e-06, 1.14950944e-06, 2.44177567e-08]], dtype=float32)

In [23]:
results = np.array(results)
results

array([[3.23959100e-08, 2.04467767e-08, 9.99982834e-01, ...,
        1.45312524e-05, 1.61152755e-07, 3.83690724e-09],
       [9.99111235e-01, 2.13342211e-09, 1.61627995e-05, ...,
        1.24111475e-05, 2.43904765e-06, 1.46191451e-04],
       [1.99100464e-06, 1.63373090e-06, 4.75807101e-05, ...,
        3.28638300e-04, 4.38523432e-03, 9.94254410e-01],
       ...,
       [4.86781448e-12, 9.25425125e-10, 1.82891835e-09, ...,
        1.19477654e-08, 1.98092039e-07, 3.68499116e-07],
       [2.73379754e-07, 5.71390729e-07, 5.15333909e-07, ...,
        3.21865431e-04, 1.64437060e-05, 9.92893875e-01],
       [9.14205671e-08, 1.01931148e-08, 9.99993205e-01, ...,
        1.69661269e-06, 1.14950944e-06, 2.44177567e-08]], dtype=float32)

In [24]:
results = np.argmax(results, axis=1)

In [25]:
results = pd.Series(results)

In [26]:
submission_csv= pd.DataFrame({'ImageId': range(1, 28001), 'Label': results})
submission_csv.to_csv('output/submission.csv', index=False)